# Introduction
In this laboratory we will get our hands dirty working with Large Language Models (e.g. GPT and BERT) to do various useful things. I you haven't already, it is highly recommended to:

+ Read the [Attention is All you Need](https://arxiv.org/abs/1706.03762) paper, which is the basis for all transformer-based LLMs.
+ Watch (and potentially *code along*) with this [Andrej Karpathy video](https://www.youtube.com/watch?v=kCc8FmEb1nY) which shows you how to build an autoregressive GPT model from the ground up.

# Exercise 1: Warming Up
In this first exercise you will train a *small* autoregressive GPT model for character generation (the one used by Karpathy in his video) to generate text in the style of Dante Aligheri. Use [this file](https://archive.org/stream/ladivinacommedia00997gut/1ddcd09.txt), which contains the entire text of Dante's Inferno (**note**: you will have to delete some introductory text at the top of the file before training). Train the model for a few epochs, monitor the loss, and generate some text at the end of training. Qualitatively evaluate the results 

In [5]:
# Your code here.
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data proeprocessing

In [57]:
from os.path import expanduser

class TextDS(Dataset):
    def __init__(self, block_size: int, path = expanduser('~') + "/datasets/commedia.txt", start: float = 0, stop: float= 0.7) -> None:
        super().__init__()
        with open(path, 'r') as f:
            self.text = f.read()
        l = len(self.text)
        self.text = self.text[int(start * l): int(stop * l)]
        self.vocab = sorted(list(set(self.text)))
        self.vocab_size = len(self.vocab)
        self.stoi = { ch: i for i, ch in enumerate(self.vocab) }
        self.itos = { i: ch for i, ch in enumerate(self.vocab) }
        self.encode = lambda s: [self.stoi[c] for c in s]
        self.decode = lambda l: ''.join([self.itos[i] for i in l])
        self.block_size = block_size

        self.data = torch.Tensor(self.encode(self.text)).type(torch.LongTensor)

    def __len__(self): return len(self.text) - (self.block_size + 1)

    def __getitem__(self, index):
        x = self.data[index: index + self.block_size]
        y = self.data[index + 1: index + self.block_size + 1]
        return x, y

In [70]:
block_size = 32
batch_size = 8
train_ds = TextDS(block_size, start=0, stop=0.3)
train_dl = DataLoader(train_ds, batch_size, True)
val_ds = TextDS(block_size, start=0.7, stop=0.9)
val_dl = DataLoader(val_ds, batch_size, True)
test_ds = TextDS(block_size, start=0.9, stop=1)
train_dl = DataLoader(train_ds, batch_size, True)

for x, y in train_dl:
    print(x.shape, y.shape)
    break


torch.Size([8, 32]) torch.Size([8, 32])


In [71]:
class AttentionHead(nn.Module):
    # take in input an embedding

    def __init__(self, input_size: int, head_size: int, block_size: int, masked: bool = True) -> None:
        super(AttentionHead, self).__init__()
        self.Q = nn.Linear(input_size, head_size)
        self.K = nn.Linear(input_size, head_size)
        self.V = nn.Linear(input_size, head_size)
        self.d = head_size
        self.masked = masked
        if self.masked:
            self.tril = torch.tril(torch.ones((block_size, block_size))).to(device)

    def forward(self, X):
        # X [B T C]
        # B, T, C = X.shape
        q = self.Q(X) # [B T D]
        k = self.K(X)
        v = self.V(X)

        qk: torch.Tensor = (q @ k.transpose(-1, -2)) / (self.d ** 0.5) # [B T D] @ [B D T] = [B T T]        
        if self.masked:
            qk = qk.masked_fill(self.tril == 0, float('-inf'))
        qk = F.dropout(qk, 0.5)
        return F.softmax(qk, dim=-1) @ v # [B T T] @ [B T D] = [B T D]
        
class MultiHeadAttention(nn.Module):

    def __init__(self, embedding_size: int, head_size: int, num_heads: int, block_size: int, masked: bool = True) -> None:
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([AttentionHead(embedding_size, head_size, block_size, masked) for _ in range(num_heads)])
        self.projection = nn.Linear(head_size * num_heads, embedding_size)

    def forward(self, X):
        # each head has an output of [B T T], stacking at [B T (T*N)]
        concat = torch.cat([head(X) for head in self.heads], dim=-1) # last dimension
        out = self.projection(concat)
        out = F.dropout(out, 0.5)
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super(FeedFoward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(0.5),
        )

    def forward(self, x):
        return self.net(x)

class TransformerBlock(nn.Module):

    def __init__(self, embedding_size, num_heads, block_size) -> None:
        super(TransformerBlock, self).__init__()
        head_size = embedding_size // num_heads
        self.mha = MultiHeadAttention(embedding_size, head_size, num_heads, block_size)
        self.feed_forward = FeedFoward(embedding_size)
        self.ln1 = nn.LayerNorm(embedding_size)
        self.ln2 = nn.LayerNorm(embedding_size)

    def forward(self, X):
        X = X + self.mha(self.ln1(X))
        X = X + self.feed_forward(self.ln2(X))
        return X
    
class BLM(nn.Module):
    def __init__(self, vocab_size: int, embedding_size: int, num_heads: int, block_size: int, num_layers: int) -> None:
        super(BLM, self).__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, embedding_size)
        self.position_embedding_table = nn.Embedding(block_size, embedding_size)
        self.blocks = nn.Sequential(*[TransformerBlock(embedding_size, num_heads, block_size) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(embedding_size) # final layer norm
        self.lm_head = nn.Linear(embedding_size, vocab_size)

    def forward(self, X):
        B, T = X.shape
        token_embedding = self.token_embedding_table(X)
        position_embedding = self.position_embedding_table(torch.arange(T, device=device))
        X = token_embedding + position_embedding
        X = self.blocks(X)
        X = self.ln_f(X)
        X = self.lm_head(X)
        return X

    def generate(self, X, max_new_token: int):
        # X is [B T]
        for _ in range(max_new_token):
            logits = self(X) # [B T C]
            # get the last timestep
            logits = logits[:, -1, :]
            # get the distribution of the next element
            probs = F.softmax(logits, dim=-1) # softmax over the last dimensiont
            next_token = torch.multinomial(probs, num_samples=1)
            X = torch.cat((X, next_token), dim=1)
        return X

# Train & Validation

In [72]:
@torch.no_grad()
def validation(model, dataloader, loss_fn):
    model.eval()
    loss = 0
    acc = 0
    for x, y in tqdm(dataloader, "Validation: ", leave=False):
        x, y = x.to(device), y.to(device)
        prediction_logits = model(x)

        B, T, C = prediction_logits.shape
        prediction_logits = prediction_logits.view(B*T, C)
        y = y.view(B*T)

        loss += loss_fn(prediction_logits, y).item()
        acc += (prediction_logits.argmax(1) == y).float().sum().item()
    return loss / len(dataloader), acc / len(dataloader.dataset)

def training(model, train_dataloader, validation_dataloader, loss_fn, optimizer, epochs, validation_freq, log):
    losses, accs = [], []
    for t in range(1, epochs + 1):
        model.train()
        for x, y in tqdm(train_dataloader, f"Epoch #{t}: ", leave=False):
            x, y = x.to(device), y.to(device)
            prediction_logits = model(x)

            B, T, C = prediction_logits.shape
            prediction_logits = prediction_logits.view(B*T, C)
            y = y.view(B*T)
            
            loss = loss_fn(prediction_logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if t % validation_freq == 0:
            lss, acc = validation(model, validation_dataloader, loss_fn)
            losses.append(lss)
            accs.append(acc)
            log_dict = {"loss": lss, "accuracy": acc}
            if log:
                wandb.log(log_dict)

    return losses, accs

In [73]:
epochs = 10
embedding_size = 512
num_head = 16
num_layers = 6
validation_freq = 5
blm = BLM(train_ds.vocab_size, embedding_size, num_head, block_size, num_layers).to(device)
optimizer = torch.optim.Adam(blm.parameters(), lr=0.001)
loss = F.cross_entropy

wandb.init(
    project="DLA Lab 2 1.0",
)
training(blm, train_dl, val_dl, loss, optimizer, epochs, validation_freq, True)
wandb.finish()

accuracy,▁▁
accuracy,1.10761
loss,nan


In [75]:
sd = blm.state_dict()
torch.save(sd, "weight.pth")

# Exercise 2: Working with Real LLMs

Our toy GPT can only take us so far. In this exercise we will see how to use the [Hugging Face](https://huggingface.co/) model and dataset ecosystem to access a *huge* variety of pre-trained transformer models.

## Exercise 2.1: Installation and text tokenization

First things first, we need to install the [Hugging Face transformer library](https://huggingface.co/docs/transformers/index):

    conda install -c huggingface -c conda-forge transformers
    
The key classes that you will work with are `GPT2Tokenizer` to encode text into sub-word tokens, and the `GPT2LMHeadModel`. **Note** the `LMHead` part of the class name -- this is the version of the GPT2 architecture that has the text prediction heads attached to the final hidden layer representations (i.e. what we need to **generate** text). 

Instantiate the `GPT2Tokenizer` and experiment with encoding text into integer tokens. Compare the length of input with the encoded sequence length.

**Tip**: Pass the `return_tensors='pt'` argument to the togenizer to get Pytorch tensors as output (instead of lists).

In [19]:
# Your code here.

## Exercise 2.2: Generating Text

There are a lot of ways we can, given a *prompt* in input, sample text from a GPT2 model. Instantiate a pre-trained `GPT2LMHeadModel` and use the [`generate()`](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to generate text from a prompt.

**Note**: The default inference mode for GPT2 is *greedy* which might not results in satisfying generated text. Look at the `do_sample` and `temperature` parameters.

In [20]:
# Your code here.

# Exercise 3: Reusing Pre-trained LLMs (choose one)

Choose **one** of the following exercises (well, *at least* one). In each of these you are asked to adapt a pre-trained LLM (`GPT2Model` or `DistillBERT` are two good choices) to a new Natural Language Understanding task. A few comments:

+ Since GPT2 is a *autoregressive* model, there is no latent space aggregation at the last transformer layer (you get the same number of tokens out that you give in input). To use a pre-trained model for a classification or retrieval task, you should aggregate these tokens somehow (or opportunistically select *one* to use).

+ BERT models (including DistillBERT) have a special [CLS] token prepended to each latent representation in output from a self-attention block. You can directly use this as a representation for classification (or retrieval).

+ The first *two* exercises below can probably be done *without* any fine-tuning -- that is, just training a shallow MLP to classify or represent with the appropriate loss function.

# Exercise 3.1: Training a Text Classifier (easy)

Peruse the [text classification datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=downloads). Choose a *moderately* sized dataset and use a LLM to train a classifier to solve the problem.

**Note**: A good first baseline for this problem is certainly to use an LLM *exclusively* as a feature extractor and then train a shallow model.

# Exercise 3.2: Training a Question Answering Model (harder)

Peruse the [multiple choice question answering datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:multiple-choice&sort=downloads). Chose a *moderately* sized one and train a model to answer contextualized multiple-choice questions. You *might* be able to avoid fine-tuning by training a simple model to *rank* the multiple choices (see margin ranking loss in Pytorch).

# Exercise 3.3: Training a Retrieval Model (hardest)

The Hugging Face dataset repository contains a large number of ["text retrieval" problems](https://huggingface.co/datasets?task_categories=task_categories:text-retrieval&p=1&sort=downloads). These tasks generally require that the model measure *similarity* between text in some metric space -- naively, just a cosine similarity between [CLS] tokens can get you pretty far. Find an interesting retrieval problem and train a model (starting from a pre-trained LLM of course) to solve it.

**Tip**: Sometimes identifying the *retrieval* problems in these datasets can be half the challenge. [This dataset](https://huggingface.co/datasets/BeIR/scifact) might be a good starting point.